In [1]:
library(SBC);
library(cmdstanr);
library(jsonlite);
library(tidyverse);
options(mc.cores = parallel::detectCores());
library(future);
plan(multisession);

options(SBC.verbose = TRUE);
options(SBC.min_chunk_size = 5);
options(cmdstanr_verbose = TRUE);

cache_dir <- "./SBC_cache"
if(!dir.exists(cache_dir)) {
    dir.create(cache_dir)
}

ALPHA <- 0.05
model_BT_1 <- cmdstanr::cmdstan_model("../models/BT_model_1.stan")
model_BT_2 <- cmdstanr::cmdstan_model("../models/BT_model_2.stan")


This is cmdstanr version 0.7.1

- CmdStanR documentation and vignettes: mc-stan.org/cmdstanr

- CmdStan path: /Users/igor.michels/.cmdstan/cmdstan-2.34.1

- CmdStan version: 2.34.1


A newer version of CmdStan is available. See ?install_cmdstan() to install it.
To disable this check set option or environment variable CMDSTANR_NO_VER_CHECK=TRUE.

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.4
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.0     v tibble    3.2.1
v lubridate 1.9.3     v tidyr     1.3.1
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter()  masks stats::filter()
x purrr::flatten() masks jsonlite::flatten()
x dplyr::lag()     masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Running make \
  /var/folders/67/qnnhzk_15ydg7t3gr6_lmx8r0000gq/T/RtmpiKQpRG/model-43763dd10f4a \
  "STANCFLAGS +=  --name='BT_model_1_model'"

--- Translating Stan model to C++ code ---
bin/stanc --name='BT_model_1_model' --o=/var/folders/67/qnnhzk_15ydg7t3gr6_lmx8r0000gq/T/RtmpiKQpRG/model-43763dd10f4a.hpp /var/folders/67/qnnhzk_15ydg7t3gr6_lmx8r0000gq/T/RtmpiKQpRG/model-43763dd10f4a.stan

--- Compiling C++ code ---
clang++ -std=c++1y -Wno-unknown-warning-option -Wno-tautological-compare -Wno-sign-compare -D_REENTRANT -Wno-ignored-attributes      -I stan/lib/stan_math/lib/tbb_2020.3/include    -O3 -I src -I stan/src -I stan/lib/rapidjson_1.1.0/ -I lib/CLI11-1.9.1/ -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.4.0 -I stan/lib/stan_math/lib/boost_1.81.0 -I stan/lib/stan_math/lib/sundials_6.1.1/include -I stan/lib/stan_math/lib/sundials_6.1.1/src/sundials    -DBOOST_DISABLE_ASSERTS          -c -include-pch stan/src/stan/model/model_header_15_0.hpp.gch -x c++ -o /var/folders/6

In [2]:
N_SIMS <- 100
N_CLUBS <- 20
N_ITER_WARMUP <- 10000
N_ITER_SAMPLING <- 10000
N_CHAINS <- 4
ADAPT_DELTA <- 0.8
ADAPT_TREE_DEPTH <- 10
REFRESH <- 100
SHOW_MESSAGES <- TRUE


In [3]:
data_generator_single_BT_1 <- function(n_clubs){
    n_seasons <- 1
    clubs <- sprintf("Club %02d", 1:n_clubs)
    clubs <- 1:n_clubs
    force <- rnorm(length(clubs))
    df <- data.frame(Club = clubs, Force = force)
    data <- merge(df, df, by = NULL) %>% filter(Club.x != Club.y)
    data$prob.home <- exp(data$Force.x) / (exp(data$Force.x) + exp(data$Force.y))
    data$home.wins <- 0

    for (i in 1:n_seasons) {
        data$random <- runif(n_clubs * (n_clubs - 1))
        data$home.wins <- data$home.wins + (data$random <  data$prob.home) * 1
    }

    data$away.wins <- n_seasons - data$home.wins
    data <- subset(data, select = -random)
    data <- subset(data, select = -prob.home)
    names(data) <- c("home_name", "home_force", "away_name", "away_force",
                     "home_wins", "away_wins")

    list(
        variables = list(
            skill = force
        ),
        generated = list(
            num_games = nrow(data),
            num_teams = n_clubs,
            team1 = data$home_name,
            team2 = data$away_name,
            team1_win = data$home_wins
        )
    )
}

In [4]:
data_generator_single_BT_2 <- function(n_clubs){
    n_seasons <- 1
    clubs <- sprintf("Club %02d", 1:n_clubs)
    clubs <- 1:n_clubs
    force <- rnorm(length(clubs))
    home_force <- rnorm(1)
    df <- data.frame(Club = clubs, Force = force)
    data <- merge(df, df, by = NULL) %>% filter(Club.x != Club.y)
    data$prob.home <- exp(data$Force.x + home_force) / (exp(data$Force.x) + exp(data$Force.y))
    data$home.wins <- 0

    for (i in 1:n_seasons) {
        data$random <- runif(n_clubs * (n_clubs - 1))
        data$home.wins <- data$home.wins + (data$random <  data$prob.home) * 1
    }

    data$away.wins <- n_seasons - data$home.wins
    data <- subset(data, select = -random)
    data <- subset(data, select = -prob.home)
    names(data) <- c("home_name", "home_force", "away_name", "away_force",
                     "home_wins", "away_wins")

    list(
        variables = list(
            skill = force,
            home_advantage = home_force
        ),
        generated = list(
            num_games = nrow(data),
            num_teams = n_clubs,
            team1 = data$home_name,
            team2 = data$away_name,
            team1_win = data$home_wins
        )
    )
}

In [5]:
set.seed(0)
data_generator_BT_1 <- SBC_generator_function(data_generator_single_BT_1, n_clubs = N_CLUBS)
dataset_BT_1 <- generate_datasets(data_generator_BT_1, N_SIMS)
backend_BT_1 <- SBC_backend_cmdstan_sample(
    model_BT_1,
    iter_warmup = N_ITER_WARMUP,
    iter_sampling = N_ITER_SAMPLING,
    chains = N_CHAINS,
    adapt_delta = ADAPT_DELTA,
    max_treedepth = ADAPT_TREE_DEPTH,
    refresh = REFRESH,
    show_messages = SHOW_MESSAGES
)

results_BT_1 <- compute_SBC(
    dataset_BT_1,
    backend_BT_1,
    keep_fit = FALSE,
    cache_mode = "results",
    cache_location = file.path(cache_dir, "results_BT_1")
)

write.csv(
    results_BT_1$stats,
    file = "SBC_cache/results_BT_1.csv"
)

Cache file exists but the backend hash differs. Will recompute.



In [ ]:
set.seed(0)
data_generator_BT_2 <- SBC_generator_function(data_generator_single_BT_2, n_clubs = N_CLUBS)
dataset_BT_2 <- generate_datasets(data_generator_BT_2, N_SIMS)
backend_BT_2 <- SBC_backend_cmdstan_sample(
    model_BT_2,
    iter_warmup = N_ITER_WARMUP,
    iter_sampling = N_ITER_SAMPLING,
    chains = N_CHAINS,
    adapt_delta = ADAPT_DELTA,
    max_treedepth = ADAPT_TREE_DEPTH,
    refresh = REFRESH,
    show_messages = SHOW_MESSAGES
)

results_BT_2 <- compute_SBC(
    dataset_BT_2,
    backend_BT_2,
    keep_fit = FALSE,
    cache_mode = "results",
    cache_location = file.path(cache_dir, "results_BT_2")
)
    
write.csv(
    results_BT_2$stats,
    file = "SBC_cache/results_BT_2.csv"
)

In [ ]:
plot_rank_hist(results_BT_1)
plot_ecdf(results_BT_1)
plot_ecdf_diff(results_BT_1)

In [ ]:
plot_rank_hist(results_BT_2)
plot_ecdf(results_BT_2)
plot_ecdf_diff(results_BT_2)

In [ ]:
results <- results_BT_1
graph <- plot_ecdf(results)
plot_data <- ggplot_build(graph)$data
confidence_interval <- plot_data[[1]]
ecdf <- plot_data[[2]]

df1 <- merge(select(confidence_interval, - c(colour, fill, group, flipped_aes, linewidth, linetype, alpha, y)),
             select(ecdf, - c(colour, fill, group, linewidth, linetype, alpha)),
             by = c("PANEL", "x"), all.x = TRUE) %>%
       group_by(PANEL, x) %>%
       summarize(ymax = max(ymax, na.rm = TRUE),
                 ymin = max(ymin, na.rm = TRUE),
                 y = max(y, na.rm = TRUE))

df1$out <- (df1$ymax < df1$y) + (df1$ymin > df1$y)
df1 <- df1 %>% group_by(PANEL) %>% summarise(out_ratio = sum(out))
df1$out_ratio <- df1$out_ratio / length(unique(ecdf$x))
df1$out <- df1$out_ratio > ALPHA

graph <- plot_ecdf_diff(results)
plot_data <- ggplot_build(graph)$data
confidence_interval <- plot_data[[1]]
ecdf <- plot_data[[2]]

df2 <- merge(select(confidence_interval, - c(colour, fill, group, flipped_aes, linewidth, linetype, alpha, y)),
             select(ecdf, - c(colour, fill, group, linewidth, linetype, alpha)),
             by = c("PANEL", "x"), all.x = TRUE) %>%
       group_by(PANEL, x) %>%
       summarize(ymax = max(ymax, na.rm = TRUE),
                 ymin = max(ymin, na.rm = TRUE),
                 y = max(y, na.rm = TRUE))

df2$out <- (df2$ymax < df2$y) + (df2$ymin > df2$y)
df2 <- df2 %>% group_by(PANEL) %>% summarise(out_ratio = sum(out))
df2$out_ratio <- df2$out_ratio / length(unique(ecdf$x))
df2$out <- df2$out_ratio > ALPHA

final_df <- merge(df1, df2, by = "PANEL", suffixes = c("", "_diff"))
c(mean(as.numeric(final_df$out)), mean(as.numeric(final_df$out_diff)))

In [ ]:
results <- results_BT_2
graph <- plot_ecdf(results)
plot_data <- ggplot_build(graph)$data
confidence_interval <- plot_data[[1]]
ecdf <- plot_data[[2]]

df1 <- merge(select(confidence_interval, - c(colour, fill, group, flipped_aes, linewidth, linetype, alpha, y)),
             select(ecdf, - c(colour, fill, group, linewidth, linetype, alpha)),
             by = c("PANEL", "x"), all.x = TRUE) %>%
       group_by(PANEL, x) %>%
       summarize(ymax = max(ymax, na.rm = TRUE),
                 ymin = max(ymin, na.rm = TRUE),
                 y = max(y, na.rm = TRUE))

df1$out <- (df1$ymax < df1$y) + (df1$ymin > df1$y)
df1 <- df1 %>% group_by(PANEL) %>% summarise(out_ratio = sum(out))
df1$out_ratio <- df1$out_ratio / length(unique(ecdf$x))
df1$out <- df1$out_ratio > ALPHA

graph <- plot_ecdf_diff(results)
plot_data <- ggplot_build(graph)$data
confidence_interval <- plot_data[[1]]
ecdf <- plot_data[[2]]

df2 <- merge(select(confidence_interval, - c(colour, fill, group, flipped_aes, linewidth, linetype, alpha, y)),
             select(ecdf, - c(colour, fill, group, linewidth, linetype, alpha)),
             by = c("PANEL", "x"), all.x = TRUE) %>%
       group_by(PANEL, x) %>%
       summarize(ymax = max(ymax, na.rm = TRUE),
                 ymin = max(ymin, na.rm = TRUE),
                 y = max(y, na.rm = TRUE))

df2$out <- (df2$ymax < df2$y) + (df2$ymin > df2$y)
df2 <- df2 %>% group_by(PANEL) %>% summarise(out_ratio = sum(out))
df2$out_ratio <- df2$out_ratio / length(unique(ecdf$x))
df2$out <- df2$out_ratio > ALPHA

final_df <- merge(df1, df2, by = "PANEL", suffixes = c("", "_diff"))
c(mean(as.numeric(final_df$out)), mean(as.numeric(final_df$out_diff)))

In [11]:
run_BT_model <- function(min_year, max_year=NULL) {
    if (is.null(max_year)) max_year <- min_year

    if (min_year == max_year) {
        output_path_1 <- paste0("../results/results_BT_1_", min_year, ".csv")
        output_path_2 <- paste0("../results/results_BT_2_", min_year, ".csv")
        input_path <- paste0('../real_data/BT_model_data_', min_year, '.json')
    } else {
        output_path_1 <- paste0("../results/results_BT_1_", min_year, "_to_", max_year, ".csv")
        output_path_2 <- paste0("../results/results_BT_2_", min_year, "_to_", max_year, ".csv")
        input_path <- paste0('../real_data/BT_model_data_', min_year, '_to_', max_year, '.json')
    }

    if (!file.exists(output_path_1)) {
        data <- read_json(input_path)
        data[["teams"]] <- NULL
        
        capture.output({
            fit_BT_1 <- suppressMessages(
                suppressWarnings(
                    model_BT_1$sample(
                        data = data,
                        chains = N_CHAINS,
                        parallel_chains = N_CHAINS,
                        refresh = 500
                    )
                )
            )
        })

        results_BT_1 <- fit_BT_1$summary(
            variables = NULL,
            posterior::default_summary_measures(),
            extra_quantiles = ~posterior::quantile2(., probs = c(.0275, .975))
        )

        write.csv(results_BT_1, file = output_path_1, row.names = FALSE)
    }


    if (!file.exists(output_path_2)) {
        data <- read_json(input_path)
        data[["teams"]] <- NULL
        
        capture.output({
            fit_BT_2 <- suppressMessages(
                suppressWarnings(
                    model_BT_2$sample(
                        data = data,
                        chains = N_CHAINS,
                        parallel_chains = N_CHAINS,
                        refresh = 500
                    )
                )
            )
        })

        results_BT_2 <- fit_BT_2$summary(
            variables = NULL,
            posterior::default_summary_measures(),
            extra_quantiles = ~posterior::quantile2(., probs = c(.0275, .975))
        )

        write.csv(results_BT_2, file = output_path_2, row.names = FALSE)
    }
}

In [ ]:
for (year in 2019:2023) {
    run_BT_model(year)
    message(paste("Completed processing for year", year))
}

run_BT_model(2019, 2023)
run_BT_model(2014, 2023)